In [1]:
import diff_classifier.aws as aws
import diff_classifier.utils as ut
import diff_classifier.msd as msd
import diff_classifier.features as ft
import diff_classifier.imagej as ij
import os
import os.path as op
import numpy as np
import numpy.ma as ma
import pandas as pd
import boto3
import skimage.io as sio

# Loading and checking a video

In [ ]:
test_image = sio.imread('P1_S1_R_0000_3_3.tif')

In [ ]:
sio.imshow(test_image[650, :, :], cmap='gray', vmin=60, vmax=250)

In [ ]:
sio.imshow(test_image[0, :, :], cmap='gray', vmin=60, vmax=250)

# Splitting images before analysis

In [2]:
prefix = 'P3_S3_R_0000'
remote_folder = "01_18_Experiment/{}".format(prefix.split('_')[0])
local_folder = os.getcwd()
ires = 512
frames = 651
filename = '{}.tif'.format(prefix)
remote_name = op.join(remote_folder, filename)
local_name = op.join(local_folder, filename)

s3 = boto3.client('s3')

names = []
for i in range(0, 4):
    for j in range(0, 4):
        names.append('{}_{}_{}.tif'.format(prefix, i, j))

try:
    for name in names:
        aws.download_s3(op.join(remote_folder, name), name)
except:
    aws.download_s3(remote_name, local_name)
    names = ij.partition_im(local_name)
    for name in names:
        #aws.upload_s3(name, op.join(remote_folder, name))
        print("Done with splitting.  Should output file of name {}".format(op.join(remote_folder, name)))

In [ ]:
ij.partition_im?

# Particle Tracking

In [3]:
for name in names:
    outfile = 'Traj_' + name.split('.')[0] + '.csv'
    local_im = op.join(local_folder, name)

    try:
        aws.download_s3(op.join(remote_folder, outfile), outfile)
    except:
        test_intensity = ij.mean_intensity(local_im)
        if test_intensity > 500:
            quality = 245
        else:
            quality = 4.5

        ij.track(local_im, outfile, template=None, fiji_bin=None, radius=4.5, threshold=0., 
              do_median_filtering=True, quality=quality, x=511, y=511, median_intensity=300.0, snr=0.0, 
              linking_max_distance=8.0, gap_closing_max_distance=10.0, max_frame_gap=2,
              track_displacement=10.0)

        aws.upload_s3(outfile, op.join(remote_folder, outfile))
    print("Done with tracking.  Should output file of name {}".format(op.join(remote_folder, outfile)))

/home/ubuntu/Fiji.app/ImageJ-linux64 --ij2 --headless --run /tmp/tmp5EnPj_.py


AttributeError: 'module' object has no attribute 'run'